In [1]:
!pip install --upgrade pymilvus

  Using cached pymilvus-2.4.9-py3-none-any.whl.metadata (5.6 kB)
  Using cached setuptools-75.4.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached grpcio-1.67.1-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached protobuf-5.28.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached environs-9.5.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached ujson-5.10.0-cp312-cp312-win_amd64.whl.metadata (9.5 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached marshmallow-3.23.1-py3-none-any.whl.metadata (7.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached numpy-2.1.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pymilvus-2.4.9-py3-none-any.whl (201 kB)
Using cached environs-9.5.0-py2.py3-none-any.whl (12 kB)
Using cached grpcio-1.67.1-cp312-cp312-win_amd64.whl (


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install git+https://github.com/zc277584121/graphrag.git

  Cloning https://github.com/zc277584121/graphrag.git to c:\users\aabid karim\appdata\local\temp\pip-req-build-q3vemd3p
  Resolved https://github.com/zc277584121/graphrag.git to commit c7263931eb9b3b6c7cd72fc11e4a6d4b11e6d48b
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached azure_identity-1.19.0-py3-none-any.whl.metadata (80 kB)
  Using cached azure_search_documents-11.5.2-py3-none-any.whl.metadata (23 kB)
  Using cached azure_storage_blob-12.23.1-py3-none-any.whl.metadata (26 kB)
  Using cached datashaper-0.0.49-py3-none-any.whl.metadata (3.7 kB)
  Using cached devtools-0.12.2-py3-none-any.whl.metadata (4.8 kB)
  Using cached fastparquet-2024.5.0-cp312-cp312-win_amd64.whl.metadata (4.3 kB)
  Us

  Running command git clone --filter=blob:none --quiet https://github.com/zc277584121/graphrag.git 'C:\Users\Aabid Karim\AppData\Local\Temp\pip-req-build-q3vemd3p'


In [1]:
import os
import nest_asyncio
import urllib.request
nest_asyncio.apply()

In [3]:
index_root = os.path.join(os.getcwd(), 'graphrag_index')
os.makedirs(os.path.join(index_root, 'input'), exist_ok=True)
url = "https://www.gutenberg.org/cache/epub/7785/pg7785.txt"
file_path = os.path.join(index_root, 'input', 'davinci.txt')
urllib.request.urlretrieve(url, file_path)
with open(file_path, 'r+', encoding='utf-8') as file:
    # We use the first 934 lines of the text file, because the later lines are not relevant for this example.
    # If you want to save api key cost, you can truncate the text file to a smaller size.
    lines = file.readlines()
    file.seek(0)
    file.writelines(lines[:934])  # Decrease this number if you want to save api key cost.
    file.truncate()

In [6]:
!python -m graphrag.index --init --root ./graphrag_index

⠋ GraphRAG Indexer 
Initializing project at ./graphrag_index
⠋ GraphRAG Indexer 


In [7]:
import yaml

settings_path = "./graphrag_index/settings.yaml"

# Load settings from the YAML file
with open(settings_path, "r") as file:
    settings = yaml.safe_load(file)



In [9]:
import os
import pandas as pd
from datetime import datetime
import hashlib
from graphrag.index.text_splitting import TokenTextSplitter

# Set up the directory structure
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
output_dir = f"./output/{timestamp}/artifacts"
os.makedirs(output_dir, exist_ok=True)

# Initialize the TokenTextSplitter for chunking
chunk_size = 1200
overlap = 100
encoding_name = "cl100k_base"
text_splitter = TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap, encoding_name=encoding_name)

# Path to your text file
input_file_path = "C:/Users/Aabid Karim/Desktop/NFQS research literature/GraphRag_implementation/graphrag_index/input/davinci.txt"

# Read the text
with open(input_file_path, "r", encoding="utf-8") as f:
    text = f.read()

# Perform chunking
chunks = text_splitter.split_text(text)

# Store each chunk's data in a list
chunk_data = []
for i, chunk in enumerate(chunks):
    chunk_id = hashlib.md5(chunk.encode()).hexdigest()
    n_tokens = len(text_splitter.encode(chunk))
    chunk_data.append({'id': chunk_id, 'chunk_text': chunk, 'n_tokens': n_tokens})

# Save to a DataFrame and write to a CSV file in the output directory
chunk_df = pd.DataFrame(chunk_data)
chunk_df.to_csv(os.path.join(output_dir, "chunked_text.csv"), index=False)

# Optional: Write chunked text files individually
for i, row in chunk_df.iterrows():
    chunk_path = os.path.join(output_dir, f"chunk_{i+1}_{row['id']}.txt")
    with open(chunk_path, "w", encoding="utf-8") as file:
        file.write(row['chunk_text'])


c:\Users\Aabid Karim\Desktop\NFQS research literature\GraphRag_implementation\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import yaml

settings_path = "./graphrag_index/settings.yaml"

# Load settings from the YAML file
with open(settings_path, "r") as file:
    settings = yaml.safe_load(file)

import os
import pandas as pd
from datetime import datetime
import hashlib
from graphrag.index.text_splitting import TokenTextSplitter

# Import utility functions from the main repo
from graphrag.index.utils.tokens import num_tokens_from_string
from graphrag.index.utils.string import clean_str
from graphrag.index.utils.hashing import gen_md5_hash

# Extract relevant settings
chunk_size = settings['chunks']['size']
overlap = settings['chunks']['overlap']
encoding_name = settings['encoding_model']

# Set up the directory structure
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
output_dir = f"./output/{timestamp}/artifacts"
os.makedirs(output_dir, exist_ok=True)

# Initialize the TokenTextSplitter for chunking
text_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=overlap,
    encoding_name=encoding_name
)

# Path to your text file
input_file_path = "C:/Users/Aabid Karim/Desktop/NFQS research literature/GraphRag_implementation/graphrag_index/input/davinci.txt"

# Get relative path from input directory
input_base_dir = settings['input']['base_dir']
relative_input_path = os.path.relpath(input_file_path, input_base_dir)

# Read and preprocess the text
with open(input_file_path, "r", encoding="utf-8") as f:
    text = f.read()

# Preprocess the text using clean_str
text = clean_str(text)

# Generate document_id as MD5 hash of the relative path
document_id = hashlib.md5(relative_input_path.encode()).hexdigest()

# Perform chunking
chunks = text_splitter.split_text(text)

# Store each chunk's data in a list
chunk_data = []
for i, chunk in enumerate(chunks):
    # Preprocess the chunk text
    chunk_text = clean_str(chunk)

    # Count tokens using num_tokens_from_string
    n_tokens = num_tokens_from_string(chunk_text, encoding_name=encoding_name)

    # Create item dictionary
    item = {
        'document_id': document_id,
        'chunk_index': i,
        'chunk_text': chunk_text,
        'n_tokens': n_tokens,
        'source': relative_input_path
    }

    # Generate chunk_id using gen_md5_hash from main repo
    item['id'] = gen_md5_hash(item, ['document_id', 'chunk_index'])

    chunk_data.append(item)

# Convert list to DataFrame
chunk_df = pd.DataFrame(chunk_data)

# Reorder columns to match main repo's output
chunk_df = chunk_df[['id', 'document_id', 'chunk_index', 'chunk_text', 'n_tokens', 'source']]

# Save to CSV
chunk_df.to_csv(os.path.join(output_dir, "chunked_text.csv"), index=False)

# Optional: Write chunked text files individually
for i, row in chunk_df.iterrows():
    chunk_path = os.path.join(output_dir, f"chunk_{i+1}_{row['id']}.txt")
    with open(chunk_path, "w", encoding="utf-8") as file:
        file.write(row['chunk_text'])


In [13]:
import yaml

settings_path = "./graphrag_index/settings.yaml"

# Load settings from the YAML file
with open(settings_path, "r") as file:
    settings = yaml.safe_load(file)

import os
import pandas as pd
from datetime import datetime
import hashlib
from graphrag.index.text_splitting import TokenTextSplitter

# Import utility functions from the main repo
from graphrag.index.utils.tokens import num_tokens_from_string
# Removed the import of clean_str
# from graphrag.index.utils.string import clean_str
from graphrag.index.utils.hashing import gen_md5_hash

# Extract relevant settings
chunk_size = settings['chunks']['size']
overlap = settings['chunks']['overlap']
encoding_name = settings['encoding_model']

# Set up the directory structure
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
output_dir = f"./output/{timestamp}/artifacts"
os.makedirs(output_dir, exist_ok=True)

# Initialize the TokenTextSplitter for chunking
text_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=overlap,
    encoding_name=encoding_name
)

# Path to your text file
input_file_path = "C:/Users/Aabid Karim/Desktop/NFQS research literature/GraphRag_implementation/graphrag_index/input/davinci.txt"

# Get relative path from input directory
input_base_dir = settings['input']['base_dir']
relative_input_path = os.path.relpath(input_file_path, input_base_dir)

# Read the text
with open(input_file_path, "r", encoding="utf-8") as f:
    text = f.read()

# Removed preprocessing with clean_str
# text = clean_str(text)

# Generate document_id as MD5 hash of the relative path
document_id = hashlib.md5(relative_input_path.encode()).hexdigest()

# Perform chunking
chunks = text_splitter.split_text(text)

# Store each chunk's data in a list
chunk_data = []
for i, chunk in enumerate(chunks):
    # Removed preprocessing with clean_str
    # chunk_text = clean_str(chunk)
    chunk_text = chunk  # Use the chunk directly

    # Count tokens using num_tokens_from_string
    n_tokens = num_tokens_from_string(chunk_text, encoding_name=encoding_name)

    # Create item dictionary
    item = {
        'document_id': document_id,
        'chunk_index': i,
        'chunk_text': chunk_text,
        'n_tokens': n_tokens,
        'source': relative_input_path
    }

    # Generate chunk_id using gen_md5_hash from main repo
    item['id'] = gen_md5_hash(item, ['document_id', 'chunk_index'])

    chunk_data.append(item)

# Convert list to DataFrame
chunk_df = pd.DataFrame(chunk_data)

# Reorder columns to match main repo's output
chunk_df = chunk_df[['id', 'document_id', 'chunk_index', 'chunk_text', 'n_tokens', 'source']]

# Save to CSV
chunk_df.to_csv(os.path.join(output_dir, "chunked_text.csv"), index=False)

# Optional: Write chunked text files individually
for i, row in chunk_df.iterrows():
    chunk_path = os.path.join(output_dir, f"chunk_{i+1}_{row['id']}.txt")
    with open(chunk_path, "w", encoding="utf-8") as file:
        file.write(row['chunk_text'])


In [15]:
import yaml

settings_path = "./graphrag_index/settings.yaml"

# Load settings from the YAML file
with open(settings_path, "r") as file:
    settings = yaml.safe_load(file)

import os
import pandas as pd
from datetime import datetime
import hashlib
from graphrag.index.text_splitting import TokenTextSplitter

# Import utility functions from the main repo
from graphrag.index.utils.tokens import num_tokens_from_string
from graphrag.index.utils.string import clean_str
from graphrag.index.utils.hashing import gen_md5_hash

# Extract relevant settings
chunk_size = settings['chunks']['size']
overlap = settings['chunks']['overlap']
encoding_name = settings['encoding_model']

# Set up the directory structure
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
output_dir = f"./output/{timestamp}/artifacts"
os.makedirs(output_dir, exist_ok=True)

# Initialize the TokenTextSplitter for chunking
text_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=overlap,
    encoding_name=encoding_name
)

# Path to your text file
input_file_path = "C:/Users/Aabid Karim/Desktop/NFQS research literature/GraphRag_implementation/graphrag_index/input/davinci.txt"

# Get relative path from input directory
input_base_dir = settings['input']['base_dir']
relative_input_path = os.path.relpath(input_file_path, input_base_dir)

# Read and preprocess the text
with open(input_file_path, "r", encoding="utf-8") as f:
    text = f.read()

# Normalize line endings
text = text.replace('\r\n', '\n').replace('\r', '\n')

# Preprocess the text using clean_str
text = clean_str(text)

# Generate document_id as MD5 hash of the relative path
document_id = hashlib.md5(relative_input_path.encode('utf-8')).hexdigest()

# Perform chunking
chunks = text_splitter.split_text(text)

# Store each chunk's data in a list
chunk_data = []
for i, chunk in enumerate(chunks):
    # Preprocess the chunk text
    chunk_text = clean_str(chunk)

    # Normalize line endings in chunk_text
    chunk_text = chunk_text.replace('\r\n', '\n').replace('\r', '\n')

    # Count tokens using num_tokens_from_string
    n_tokens = num_tokens_from_string(chunk_text, encoding_name=encoding_name)

    # Create item dictionary
    item = {
        'document_id': document_id,
        'chunk_index': i,
        'chunk_text': chunk_text,
        'n_tokens': n_tokens,
        'source': relative_input_path
    }

    # Generate chunk_id using gen_md5_hash from main repo
    item['id'] = gen_md5_hash(item, ['document_id', 'chunk_index'])

    # For debugging: print intermediate values
    print(f"Document ID: {document_id}")
    print(f"Chunk Index: {i}")
    print(f"Chunk Text Hash Input: {item['document_id']}{item['chunk_index']}")
    print(f"Chunk ID: {item['id']}")

    chunk_data.append(item)

# Convert list to DataFrame
chunk_df = pd.DataFrame(chunk_data)

# Reorder columns to match main repo's output
chunk_df = chunk_df[['id', 'document_id', 'chunk_index', 'chunk_text', 'n_tokens', 'source']]

# Save to CSV
chunk_df.to_csv(os.path.join(output_dir, "chunked_text.csv"), index=False)

# Optional: Write chunked text files individually
for i, row in chunk_df.iterrows():
    chunk_path = os.path.join(output_dir, f"chunk_{i+1}_{row['id']}.txt")
    with open(chunk_path, "w", encoding="utf-8") as file:
        file.write(row['chunk_text'])


Document ID: a92662dd22def269a1ab5f855fa4e00a
Chunk Index: 0
Chunk Text Hash Input: a92662dd22def269a1ab5f855fa4e00a0
Chunk ID: a3fa966077fc03166ac5b860b4db07db
Document ID: a92662dd22def269a1ab5f855fa4e00a
Chunk Index: 1
Chunk Text Hash Input: a92662dd22def269a1ab5f855fa4e00a1
Chunk ID: c339bbcd7a05e7e6fa9846c4300370f4
Document ID: a92662dd22def269a1ab5f855fa4e00a
Chunk Index: 2
Chunk Text Hash Input: a92662dd22def269a1ab5f855fa4e00a2
Chunk ID: 2c7c51c9cb3d001ad0e27bb1bd4995ef
Document ID: a92662dd22def269a1ab5f855fa4e00a
Chunk Index: 3
Chunk Text Hash Input: a92662dd22def269a1ab5f855fa4e00a3
Chunk ID: bba595619694a96aea37ab8ad39010a8
Document ID: a92662dd22def269a1ab5f855fa4e00a
Chunk Index: 4
Chunk Text Hash Input: a92662dd22def269a1ab5f855fa4e00a4
Chunk ID: 8fdd31800b31b150978a7faa505e8691
Document ID: a92662dd22def269a1ab5f855fa4e00a
Chunk Index: 5
Chunk Text Hash Input: a92662dd22def269a1ab5f855fa4e00a5
Chunk ID: b5f7146cc0d701007e04fea81254c31b
Document ID: a92662dd22def269a1ab5

In [6]:
import yaml
import os
import re
from pathlib import Path
import pandas as pd
from datetime import datetime

# Load settings from settings.yml
settings_path = r"C:\Users\Aabid Karim\Desktop\NFQS research literature\GraphRag_implementation\graphrag_index\settings.yaml"
with open(settings_path, "r") as file:
    settings = yaml.safe_load(file)

from graphrag.index.text_splitting import TokenTextSplitter
from graphrag.index.utils.hashing import gen_md5_hash

# Extract settings
chunk_size = settings['chunks']['size']
overlap = settings['chunks']['overlap']
encoding_name = settings['encoding_model']
input_base_dir = settings['input']['base_dir']
file_pattern = settings['input']['file_pattern']
output_dir = settings['storage']['base_dir']
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
output_dir = output_dir.replace('${timestamp}', timestamp)
os.makedirs(output_dir, exist_ok=True)

# Initialize TokenTextSplitter
text_splitter = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=overlap,
    encoding_name=encoding_name
)

# Specify your input file
# Replace 'your_document.txt' with the name of your input file
input_file_name = r'C:\Users\Aabid Karim\Desktop\NFQS research literature\GraphRag_implementation\graphrag_index\input\davinci.txt'
input_file_path = os.path.join(input_base_dir, input_file_name)

# Check if the file exists
if not os.path.isfile(input_file_path):
    raise FileNotFoundError(f"Input file not found: {input_file_path}")

# Read the text from the input file
with open(input_file_path, "r", encoding="utf-8") as f:
    text = f.read()

# Normalize line endings
text = text.replace('\r\n', '\n').replace('\r', '\n')

# You can remove leading/trailing whitespace if desired
text = text.strip()

# Extract metadata (if applicable)
group = {}  # You can populate this with metadata if needed

# Generate document_id
new_item = {**group, "text": text}
new_item["id"] = gen_md5_hash(new_item, new_item.keys())
document_id = new_item["id"]

# Split the text into chunks
chunks = text_splitter.split_text(text)

# Process each chunk
chunk_data = []
for i, chunk_text in enumerate(chunks):
    # Normalize line endings in chunk_text
    chunk_text = chunk_text.replace('\r\n', '\n').replace('\r', '\n')

    # Remove leading/trailing whitespace if desired
    chunk_text = chunk_text.strip()

    # Count tokens in the chunk
    n_tokens = text_splitter.num_tokens(chunk_text)

    # Create item for hashing
    item = {
        'chunk_text': chunk_text,
    }

    # Generate chunk_id
    chunk_id = gen_md5_hash(item, ['chunk_text'])

    chunk_data.append({
        'id': chunk_id,
        'document_id': document_id,
        'chunk_index': i,
        'chunk_text': chunk_text,
        'n_tokens': n_tokens,
        'source': os.path.relpath(input_file_path, input_base_dir),
    })

# Create DataFrame and save
chunk_df = pd.DataFrame(chunk_data)
chunk_df = chunk_df[['id', 'document_id', 'chunk_index', 'chunk_text', 'n_tokens', 'source']]
chunk_df.to_csv(os.path.join(output_dir, "chunked_text.csv"), index=False)

# Optional: Print the DataFrame
print(chunk_df)


c:\Users\Aabid Karim\Desktop\NFQS research literature\GraphRag_implementation\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                 id                       document_id  \
0  f4fb901f94f91d0c20e620b2f0a38f76  fe01289c26eebdce9847e2a708e047a4   
1  16e67091f829636cc7c7f93b1544e84b  fe01289c26eebdce9847e2a708e047a4   
2  35f3567b12f68570274aba8002932720  fe01289c26eebdce9847e2a708e047a4   
3  2bbe73c39670e59a04f86f499ef8f02c  fe01289c26eebdce9847e2a708e047a4   
4  3de06150e0b5dd11f7486f323c10f4e4  fe01289c26eebdce9847e2a708e047a4   
5  3b0f8d2c812880568dffb798034e2d52  fe01289c26eebdce9847e2a708e047a4   
6  845361b47864c99ceaaa1de7e4804f49  fe01289c26eebdce9847e2a708e047a4   
7  5a497f914061af875d43a4a9879bfa93  fe01289c26eebdce9847e2a708e047a4   
8  9a41c21ad6ee6a73b65789a1f1a17b5c  fe01289c26eebdce9847e2a708e047a4   
9  e3e06493a5ae7412a0ef1ecbfde3bd0f  fe01289c26eebdce9847e2a708e047a4   

   chunk_index                                         chunk_text  n_tokens  \
0            0  ﻿The Project Gutenberg eBook of Leonardo Da Vi...      1200   
1            1  by his grandfather\nAn

Splitting on stentence method

In [7]:
import yaml
import os
import re
from pathlib import Path
import pandas as pd
from datetime import datetime
import nltk
nltk.download('punkt')

# Load settings from settings.yml
settings_path = "C:/Users/Aabid Karim/Desktop/NFQS research literature/GraphRag_implementation/graphrag_index/settings.yaml"
with open(settings_path, "r") as file:
    settings = yaml.safe_load(file)

from graphrag.index.utils.hashing import gen_md5_hash

# Extract settings
encoding_name = settings['encoding_model']
input_base_dir = settings['input']['base_dir']
file_pattern = settings['input']['file_pattern']
output_dir = settings['storage']['base_dir']
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
output_dir = output_dir.replace('${timestamp}', timestamp)
os.makedirs(output_dir, exist_ok=True)

# Specify your input file
input_file_name = 'C:/Users/Aabid Karim/Desktop/NFQS research literature/GraphRag_implementation/graphrag_index/input/davinci.txt'
input_file_path = input_file_name  # Since we have the full path

# Check if the file exists
if not os.path.isfile(input_file_path):
    raise FileNotFoundError(f"Input file not found: {input_file_path}")

# Read the text from the input file
with open(input_file_path, "r", encoding="utf-8") as f:
    text = f.read()

# Normalize line endings
text = text.replace('\r\n', '\n').replace('\r', '\n')

# Remove leading/trailing whitespace if desired
text = text.strip()

# Extract metadata (if applicable)
group = {}  # You can populate this with metadata if needed

# Generate document_id
new_item = {**group, "text": text}
new_item["id"] = gen_md5_hash(new_item, new_item.keys())
document_id = new_item["id"]

# Split the text into sentences using NLTK
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(text)

# Process each sentence as a chunk
chunk_data = []
for i, chunk_text in enumerate(sentences):
    # Normalize line endings in chunk_text
    chunk_text = chunk_text.replace('\r\n', '\n').replace('\r', '\n')

    # Remove leading/trailing whitespace if desired
    chunk_text = chunk_text.strip()

    # Count tokens in the chunk (optional)
    # You can use your tokenization method here if needed
    # For simplicity, we'll count words as tokens
    n_tokens = len(chunk_text.split())

    # Create item for hashing
    item = {
        'chunk_text': chunk_text,
    }

    # Generate chunk_id
    chunk_id = gen_md5_hash(item, ['chunk_text'])

    chunk_data.append({
        'id': chunk_id,
        'document_id': document_id,
        'chunk_index': i,
        'chunk_text': chunk_text,
        'n_tokens': n_tokens,
        'source': os.path.relpath(input_file_path, input_base_dir),
    })

# Create DataFrame and save
chunk_df = pd.DataFrame(chunk_data)
chunk_df = chunk_df[['id', 'document_id', 'chunk_index', 'chunk_text', 'n_tokens', 'source']]
chunk_df.to_csv(os.path.join(output_dir, "chunked_text.csv"), index=False)

# Optional: Print the DataFrame
print(chunk_df)


[nltk_data] Downloading package punkt to C:\Users\Aabid
[nltk_data]     Karim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


                                   id                       document_id  \
0    dbbaa31a182ce8bf616765e9c417082c  fe01289c26eebdce9847e2a708e047a4   
1    48a99c74d7733d5de771727c6df5fb6d  fe01289c26eebdce9847e2a708e047a4   
2    3d07f1b8e7a48ae00e44a9dbf55907fd  fe01289c26eebdce9847e2a708e047a4   
3    ef1123ec0166d28c5d2ca5bb20b59aa6  fe01289c26eebdce9847e2a708e047a4   
4    34eee1a5e8560bdd87fa2083461a4f07  fe01289c26eebdce9847e2a708e047a4   
..                                ...                               ...   
266  1551a0391565f344b903666bb52cbadb  fe01289c26eebdce9847e2a708e047a4   
267  71137885ef4b4150c24e7c9129427c6d  fe01289c26eebdce9847e2a708e047a4   
268  a555e082bbcd21a8c097cd3fa495db66  fe01289c26eebdce9847e2a708e047a4   
269  f2c571599d1e6b2d93278ec1c7cc60cc  fe01289c26eebdce9847e2a708e047a4   
270  66c229e38d9eaf4c53e44b00b3d049e8  fe01289c26eebdce9847e2a708e047a4   

     chunk_index                                         chunk_text  n_tokens  \
0              0  